<a href="https://colab.research.google.com/github/Danddt/Russian_Opinion_Polls/blob/main/Copy_of_scrapping_NEWS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
urls=['https://thehackernews.com/2023/11/konni-group-using-russian-language.html',
'https://thehackernews.com/2023/11/russian-cyber-espionage-group-deploys.html',
'https://thehackernews.com/2023/11/russian-hackers-launch-largest-ever.html',
'https://thehackernews.com/2023/11/stealthy-kamran-spyware-targeting-urdu.html',
'https://thehackernews.com/2023/11/stripedfly-malware-operated-unnoticed.html',
'https://thehackernews.com/2023/10/yorotrooper-researchers-warn-of.html',
'https://thehackernews.com/2023/10/ex-nsa-employee-pleads-guilty-to.html',
'https://thehackernews.com/2023/10/exelastealer-new-low-cost-cybercrime.html',
'https://thehackernews.com/2023/10/sophisticated-mata-framework-strikes.html',
'https://thehackernews.com/2023/10/tetrisphantom-cyber-espionage-via.html',
'https://thehackernews.com/2023/10/pro-russian-hackers-exploiting-recent.html',
'https://thehackernews.com/2023/09/mysterious-sandman-threat-actor-targets.html',
'https://thehackernews.com/2023/09/how-cyberattacks-are-transforming.html',
'https://thehackernews.com/2023/09/north-korean-hackers-exploit-zero-day.html',
'https://thehackernews.com/2023/09/russian-state-backed-infamous-chisel.html',
'https://thehackernews.com/2023/08/russian-hackers-use-zulip-chat-app-for.html',
'https://www.resecurity.com/blog/article/ransomware-attacks-against-the-energy-sector-on-the-rise-nuclear-and-oil-gas-are-major-targets-2024',]

In [ ]:
def get_info(url):
  response= requests.get(url).content
  soup = BeautifulSoup(response, "html.parser")
  return soup

In [ ]:
def read_url(url):
  response= requests.get(url).content
  soup = BeautifulSoup(response, "html.parser")
  art={}
  para_list=[]
  for para in soup.find_all('p'):
      all_para = para.get_text()
      para_list.append(all_para)

  art[url] = ' '. join(para_list)
  return art

In [ ]:
dic_list = []
for urlo in urls:
  temp = read_url(urlo)
  df = pd.DataFrame.from_dict(temp, orient='index')

  dic_list.append(df)

In [ ]:
art_df = pd.concat(dic_list).reset_index()

In [ ]:
art_df.columns=['web', 'content']

In [ ]:
art_df

,web,content
0,https://thehackernews.com/2023/11/konni-group-...,A new phishing attack has been observed levera...
1,https://thehackernews.com/2023/11/russian-cybe...,Russian cyber espionage actors affiliated with...
2,https://thehackernews.com/2023/11/russian-hack...,Russian threat actors have been possibly linke...
3,https://thehackernews.com/2023/11/stealthy-kam...,Urdu-speaking readers of a regional news websi...
4,https://thehackernews.com/2023/11/stripedfly-m...,An advanced strain of malware masquerading as ...
5,https://thehackernews.com/2023/10/yorotrooper-...,A relatively new threat actor known as YoroTro...
6,https://thehackernews.com/2023/10/ex-nsa-emplo...,A former employee of the U.S. National Securit...
7,https://thehackernews.com/2023/10/exelastealer...,A new information stealer named ExelaStealer h...
8,https://thehackernews.com/2023/10/sophisticate...,An updated version of a sophisticated backdoor...
9,https://thehackernews.com/2023/10/tetrisphanto...,Government entities in the Asia-Pacific (APAC)...


In [ ]:
import torch

import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest

In [ ]:
# source code: https://www.activestate.com/blog/how-to-do-text-summarization-with-python/
def summarize(text, per):
    nlp = spacy.load('en_core_web_sm')
    nlp.max_length = 8457792 # or higher
    doc= nlp(text)
    tokens=[token.text for token in doc]
    word_frequencies={}
    for word in doc:
        if word.text.lower() not in list(STOP_WORDS):
            if word.text.lower() not in punctuation:
                if word.text not in word_frequencies.keys():
                    word_frequencies[word.text] = 1
                else:
                    word_frequencies[word.text] += 1
    max_frequency=max(word_frequencies.values())
    for word in word_frequencies.keys():
        word_frequencies[word]=word_frequencies[word]/max_frequency
    sentence_tokens= [sent for sent in doc.sents]
    sentence_scores = {}
    for sent in sentence_tokens:
        for word in sent:
            if word.text.lower() in word_frequencies.keys():
                if sent not in sentence_scores.keys():
                    sentence_scores[sent]=word_frequencies[word.text.lower()]
                else:
                    sentence_scores[sent]+=word_frequencies[word.text.lower()]
    select_length=int(len(sentence_tokens)*per)
    summary=nlargest(select_length, sentence_scores,key=sentence_scores.get)
    final_summary=[word.text for word in summary]
    summary=''.join(final_summary)
    summary =summary.replace('\n', '')
    return summary

In [ ]:
summer =[]
for item in art_df['content']:
  x =summarize(item, 0.1)
  summer.append(x)

In [ ]:
art_df['summery']= summer

In [ ]:
art_df.to_csv('art_read_summery.csv', index=False)

In [ ]:
art_df

,web,content,summery
0,https://thehackernews.com/2023/11/konni-group-...,A new phishing attack has been observed levera...,The disclosure also arrives less than two week...
1,https://thehackernews.com/2023/11/russian-cybe...,Russian cyber espionage actors affiliated with...,"Check Point, which detailed Gamaredon's (aka A..."
2,https://thehackernews.com/2023/11/russian-hack...,Russian threat actors have been possibly linke...,Russian threat actors have been possibly linke...
3,https://thehackernews.com/2023/11/stealthy-kam...,Urdu-speaking readers of a regional news websi...,"""As this malicious app has never been offered ..."
4,https://thehackernews.com/2023/11/stripedfly-m...,An advanced strain of malware masquerading as ...,"""The malware payload itself is structured as a..."
5,https://thehackernews.com/2023/10/yorotrooper-...,A relatively new threat actor known as YoroTro...,In addition to experimenting with multiple typ...
6,https://thehackernews.com/2023/10/ex-nsa-emplo...,A former employee of the U.S. National Securit...,"""Dalke admitted that between August and Septem..."
7,https://thehackernews.com/2023/10/exelastealer...,A new information stealer named ExelaStealer h...,"Earlier this week, U.S. cybersecurity and inte..."
8,https://thehackernews.com/2023/10/sophisticate...,An updated version of a sophisticated backdoor...,"""Attackers used many techniques to hide their ..."
9,https://thehackernews.com/2023/10/tetrisphanto...,Government entities in the Asia-Pacific (APAC)...,"""The attacker covertly spied on and harvested ..."
